In [17]:
%pip install google-auth-oauthlib google-api-python-client

  Using cached google_auth_oauthlib-1.2.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.3.1-py3-none-any.whl.metadata (7.9 kB)
Using cached google_auth_oauthlib-1.2.2-py3-none-any.whl (19 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
Using cached oauthlib-3.3.1-py3-none-any.whl (160 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [google-auth-oauthlib]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import spotipy
import os
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import pprint
import threading

In [3]:
load_dotenv()

client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
redirect_uri = os.getenv("SPOTIFY_REDIRECT_URI")
playlist_id = os.getenv("SPOTIFY_PLAYLIST_ID")

scope = "user-library-read"

In [4]:
def get_playlist_tracks(playlist_id):
    names = []
    # Create an OAuth object --> gives us access to the Spotify API
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id, 
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope=scope
    ))

    results = sp.playlist(playlist_id)
    
    tracks = results['tracks']['items']
    
    for item in tracks:
        names.append(item['track']['name'])
    
    return names

In [5]:
# Crack playlist ID: 3xWx987zAhFtL1r5kPhtpr

crack = get_playlist_tracks("3xWx987zAhFtL1r5kPhtpr")
print(crack)

['Rodeo (Remix)', 'One Of The Girls (with JENNIE, Lily Rose Depp)', 'Confident', 'Her Way', 'Belong to the City', 'PERSIAN RUGS', 'FREAK IN YOU', 'Earned It (Fifty Shades Of Grey)', 'Streets', 'Wine Pon You (feat. Konshens)', 'Girls Need Love (with Drake) - Remix', "Don't", 'Wus Good / Curious', 'Coming Down', 'The Party & The After Party', 'What You Need', 'Sativa', 'Feel It', 'Sex With Me', 'Needed Me', 'Hotel', 'Woo', 'Lost in the Fire (feat. The Weeknd)', 'Often', 'Into It', "Sun Down, I'm Up - Sped Up & Pitched", 'Call Out My Name', 'Cherry Hill', 'All The Time', 'Excitement', 'One Of The Girls (with JENNIE, Lily Rose Depp)', 'Wine Pon You (feat. Konshens)', 'Break from Toronto', "Don't Run", 'Dreamin', 'SOMETHING ABOUT YOU', 'TBH', 'GREEDY', 'Come and See Me (feat. Drake)', 'Recognize (feat. Drake)', 'THINGS & SUCH', 'Say It', "Let Em' Know", 'Poison', 'Been Away', 'Forever Yours', 'Chanel', 'R e s e n t m e n t', 'N o C h i l l', 'Stay Ready (What A Life)', 'Grown Woman', 'WEST 

In [6]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [7]:
# Create the flow using the client secrets file from the Google API Console.
flow = InstalledAppFlow.from_client_secrets_file( 
    'client_secret.json',
    scopes=['https://www.googleapis.com/auth/youtube.force-ssl']) # This grants full access to the YouTube account

""" Essentially, flow will get an authentication code and will use that alongside the client_secrets.json file to retrieve an 
access token that allows our app to get what's defined by the scopes"""


credentials = flow.run_local_server(port=8081) # Store the access and refresh tokens in the credentials object

youtube = build('youtube', 'v3', credentials=credentials) # build the YouTube service object; gives access to YouTube API 

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=175375944583-k4t537kob18t0tt5621dbm0rhm8q69u4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8081%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=z51WVVWBEfoB2wcFhEQPRPZT7qEyB9&access_type=offline


In [9]:
def search_song(query):
    result = youtube.search().list(part="snippet", q=query, type="video", maxResults=1).execute()
    # result['items'][0]['snippet']['title'] --> title of song
    return result

# pprint.pprint(search_song("Some of Your Love PND Slowed Reverb"))

In [13]:
# slowed_version creates a list of dictionaries, each filled with search_song() output of indices
# Test with only first 3 songs

def main():
    threads = []
    slowed_version = []
    for i in range(len(crack[:3])):
        t = threading.Thread(target=search_song, args=((crack[i] + " Slowed Reverb"),))
        slowed_version.append(search_song(crack[i] + " Slowed Reverb"))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()
    return slowed_version

In [8]:
def create_playlist(title, description=None, privacyStatus='private', defaultLanguage=None):
    snippet_body = {
        'snippet': {
            'title': title,
            'description': description,
            'defaultLanguage': defaultLanguage
        },
        'status': {
            'privacyStatus': privacyStatus
        }
    }
    return youtube.playlists().insert(part="snippet,status", body=snippet_body).execute()

In [ ]:
result = create_playlist(title="Test")
# result['id'] is the playlist ID

In [1]:
def add_songs(videoID, playlistID):
    snippet_body = {
        "snippet": {
            "playlistID" : playlistID,
            "resourceID": {
                "kind": "youtube#video",
                "videoID": videoID
            }
        }
    }
    # This makes a POST request to the YouTube API
    return youtube.playlistItems().insert(part="snippet", body=snippet_body).execute()